In [ ]:
import requests
resp = requests.get("https://candidate.hubteam.com/candidateTest/v3/problem/dataset?userKey=d766f318b666a0627ad47ed7a41d")

In [ ]:
if resp.status_code == 200:
  data = resp.json()['callRecords']

In [ ]:
def get_records_by_customer(data):
  records_by_customer = {}
  for _data in data:
    customer = _data['customerId']
    if records_by_customer.get(customer, None) is None:
      records_by_customer[customer] = []
    records_by_customer[customer].append(_data)
  return records_by_customer

records_by_customer = get_records_by_customer(data)

In [ ]:
data[0]

{'customerId': 79480,
 'callId': '55a27187-538a-4f57-a3be-4dd5be2153d1',
 'startTimestamp': 1704974700000,
 'endTimestamp': 1704975600000}

In [ ]:
def update_calls(calls, ts, call_id):
  _ts = datetime.datetime.fromtimestamp(ts)
  y = _ts.year
  m = _ts.month
  d = _ts.day
  h = _ts.hour
  mi = _ts.minute
  s = _ts.second

  ts_str_1 = str(y) + ":" + str(m) + ":" + str(d)
  ts_str_2 = str(h) + ":" + str(mi) + ":" + str(s)

  if calls.get(ts_str_1, None) is None:
    calls[ts_str_1] = {}
  if calls[ts_str_1].get(ts_str_2, None) is None:
    calls[ts_str_1][ts_str_2] = []
  calls[ts_str_1][ts_str_2].append(call_id)

  return calls

In [ ]:
import datetime
def parse_ts(records_by_customer):

  ts_delta = datetime.timedelta(seconds=1)
  result = {}
  for customer in records_by_customer:
    calls = {}
    records = records_by_customer[customer]
    for record in records:
      call_id = record['callId']
      startTimestamp = record['startTimestamp']/1000.0
      endTimestamp = record['endTimestamp']/1000.0

      i = startTimestamp
      while i < endTimestamp:
        calls = update_calls(calls, i, call_id)
        i += 1

    for theday in calls.keys():
      num_conns = 0
      for thetime in calls[theday]:
        if len(calls[theday][thetime]) > num_conns:
          if result.get(customer, None) is None:
            result[customer] = {}
          if result[customer].get(theday, None) is None:
            result[customer][theday] = []
          result[customer][theday] = [thetime, len(calls[theday][thetime]), calls[theday][thetime]]
          num_conns = len(calls[theday][thetime])
  return result
result = parse_ts(records_by_customer)

In [ ]:
import re
final_result = {}
final_result['summary'] = []
for customer in result:
  for theday in result[customer]:
    thetime, count, calls = result[customer][theday]

    m_obj = re.match("([0-9]+):([0-9]+):([0-9]+)", theday)
    y = int(m_obj.group(1))
    m = int(m_obj.group(2))
    d = int(m_obj.group(3))

    m_obj = re.match("([0-9]+):([0-9]+):([0-9]+)", thetime)
    h = int(m_obj.group(1))
    mi = int(m_obj.group(2))
    s = int(m_obj.group(3))

    _parsed_ts = datetime.datetime(year=y, month=m, day=d, hour=h, minute=mi, second=s)
    parsed_ts = _parsed_ts.strftime('%b %d %Y %H:%M:%S')

    data = {
        "customer": customer,
        "Date": parsed_ts,
        "Concurrent calls": count,
        "Call IDs": calls
    }
    final_result['summary'].append(data)

In [ ]:
import pprint
pprint.pprint(final_result)

{'summary': [{'Call IDs': ['55a27187-538a-4f57-a3be-4dd5be2153d1',
                           '8672a69f-7d88-4c67-8db1-15988582fe8b',
                           '467d0f90-71d7-4edf-8221-47d7dd29339f',
                           '43c20484-0e4d-4c12-b6af-216f190f7d7a',
                           '30f8ebbe-e383-40d6-98b5-7b7a430a0206',
                           'b3a4057b-d9cd-4aa7-ab29-394dd79559d8',
                           'fae27d32-ee30-4c81-b68d-6da6663b8e8c',
                           'f21ba37c-7c7d-4382-b04b-f75269297c73',
                           'bad8183b-b936-4c6d-9e07-e29afeaacee7',
                           'b6b44863-b54b-4461-b00f-ad0100e87ff6'],
              'Concurrent calls': 10,
              'Date': 'Jan 11 2024 12:18:00',
              'customer': 79480},
             {'Call IDs': ['d54c3885-3c49-4743-a7ee-9b2c1bfae34f',
                           '119486ca-31c1-4f71-abd3-4546d1355fc9',
                           '9c2a7774-02e4-4c2f-958e-934da5ccd566',
          

In [ ]:
# Code stub for the POST (need to replace the url with the correct endpoint)
import json
data_json = json.dumps(final_result)
headers = {'Content-type': 'application/json'}
url = 'https://<this would be the post URL>'
response = requests.post(url, data=data_json, headers=headers)
assert response.status_code == 200